In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
chemin = "/content/drive/MyDrive/Stage Defense"


Mounted at /content/drive


In [ ]:
!pip install spacy-transformers
!python3 -m spacy download en_core_web_trf

     |████████████████████████████████| 6.4 MB 6.5 MB/s 
     |████████████████████████████████| 998 kB 66.2 MB/s 
     |████████████████████████████████| 456 kB 71.3 MB/s 
     |████████████████████████████████| 2.2 MB 63.4 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 621 kB 45.6 MB/s 
     |████████████████████████████████| 10.1 MB 45.3 MB/s 
     |████████████████████████████████| 895 kB 34.4 MB/s 
     |████████████████████████████████| 3.3 MB 41.3 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninsta

In [ ]:
import re

import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.corpus import stopwords
import spacy

import gensim
import gensim.downloader

In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
nlp = spacy.load('en_core_web_trf')
cheminmodeleW2V = chemin+"/Ressources/frWac_no_postag_no_phrase_500_cbow_cut100.bin"
model_w2v  = gensim.downloader.load('fasttext-wiki-news-subwords-300')


In [ ]:
nltk.download('stopwords')
# stopwords de nltk
nltk_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
chemin = "/content/drive/MyDrive/Stage Defense"
chemindata = chemin+"/Corpus/Toutes_mentions_Defense_EN.csv"

# Pré-traitement corpus

In [ ]:
df = pd.read_csv(chemindata,sep=",", index_col=0, keep_default_na=False,na_values=["NA"])

In [ ]:
df = df[~(df.cible_contexte.isin([""]))&(~df.procédé_de_nommage.isin(["",np.NAN]))]

mention

# word2vec

mention

In [ ]:
def cree_vecteurs_designations(df):
  liste_vecteurs_m = []
  for mention in df.mention:
    doc = nlp(mention)
    if "défense" in mention.lower():
      print("BRUT",mention)
    lemmes = [t.lemma_ for t in doc if t.lemma_ not in nltk_stopwords|{"défense","defense"}]
    array_vec = np.array([model_frWac[lem] for lem in lemmes if (lem in model_frWac.vocab )])
    #print(array_vec.shape)
    if not array_vec.any():
      array_vec = np.zeros((1,500))
    array_moy = np.mean(array_vec,axis=0)
    liste_vecteurs_m.append(array_moy)
  return liste_vecteurs_m 


In [ ]:
liste_vecteurs_m = cree_vecteurs_designations(df)
df_mention_w2v =  pd.DataFrame(liste_vecteurs_m,index=df.index)

In [ ]:
df_w2v = df_mention_w2v

In [ ]:
 df_w2v.columns = list(range(len(df_w2v.columns)))

In [ ]:
df_w2v.to_csv(chemin+"/Corpus/W2V_EN_mentions_Defense.csv",sep=",")

#Features generation

In [ ]:

import gensim
from sklearn.metrics.pairwise import cosine_similarity 
import re

In [ ]:
def moyenne_vecteurs(liste_mots):
  liste_vecteurs = []
  for mot in liste_mots:
    if mot in model_w2v.vocab:
      liste_vecteurs.append(model_w2v[mot])
  print(len(liste_vecteurs))
  return np.mean(np.array(liste_vecteurs),axis=0)

In [ ]:
def normalise(char):
  char = re.sub("[],\-\n;&]", " " , char)
  return char.lower()

In [ ]:
def normalise_mention(men):
  #men = men.lower()
  men = re.sub("^-","",men)
  men = men.replace("("," (")
  men = men.replace(")",")")
  return re.sub("\s+"," ",men).strip()

In [ ]:
def detecte_norme(mention):
  stopwords = nltk_stopwords | {"la","La"}
  if "défense" in mention.lower() or "defense" in mention.lower():
    doc = nlp(normalise(mention))
    #print(mention)
    lemmes = [t.lemma_ for t in doc if t.lemma_ not in stopwords and re.match("\w+",t.lemma_)]
    #print(lemmes)
    if lemmes in  [["défense"],["defense"]]:
      return 1
  return 0

In [ ]:
def cos_sim(a,b):
  return cosine_similarity(a.reshape(1, -1),b.reshape(1, -1))[0][0]

In [ ]:
def contient_ordinal(mention):
  
  if ordinal.search(mention.lower()):
    return 1
  return 0

In [ ]:
def contient_superlatif(mention):
  
  if any(super in mention.lower() for super in set_superlatifs) or any([superlatif.match(t.text) for t in nlp(mention) if t.pos_ == 'ADJ']):
    return 1
  return 0

In [ ]:
def contient_lieu_geo(mention,vecteur_lieu,vecteur_geo):
  #if True or "laboratoire" not in mention and "mélange" not in mention: return 0,0,0,0,0
  #ajouter est type de lieu. Seuil = 0.30 Regarder tous les noms
  doc = nlp(mention)
  #print(doc[1].dep_)
  lieu,geo = 0,0
  sim_tete_lieu, sim_tete_geo,sim_tete_norme = 0,0,0
  for tok in doc:
    #print(tok,tok.pos_, tok.dep_, tok.tag_)
    lemme = tok.text
    
    if lemme in model_w2v:
      sim_lieu = cos_sim(model_w2v[lemme],vecteur_lieu)
      sim_geo = cos_sim(model_w2v[lemme],vecteur_geo)
      #print(lemme, sim_lieu,sim_geo)

      if (tok.dep_ == "ROOT" and all([tok.dep_ != "appos" for tok in doc ])) or tok.dep_ == "appos":
        #print(tok,tok.dep_, "|",mention)
        sim_tete_lieu = sim_lieu
        sim_tete_geo = sim_geo
        sim_tete_norme = cos_sim(model_w2v[lemme],model_w2v["Défense"])
      if lemme.lower() in ["the"]:continue
      if sim_geo > 0.55 :
        geo = 1

      if sim_lieu > 0.55 and tok.pos_ in {"NOUN","PROPN"}:
        lieu = 1


  if sim_tete_lieu == 0:
    print(mention, [t for t in doc if t.dep_ == "ROOT"])
  return lieu,geo, sim_tete_lieu, sim_tete_geo,sim_tete_norme


In [ ]:
def contient_guillemet(mention):

  return int('"' in mention or '»' in mention or '«' in mention)

In [ ]:
set_superlatifs = {"most","best","least","worst","favourite"}

In [ ]:
superlatif = re.compile("\w+est$")

In [ ]:
ordinal = re.compile("(first)|(second)|(third)((four)|((fif)|(six)|(seven)|(eigh)|(nine?)|(ten)|(eleven)|(twelf)|(teen)|(tie)|(hundred)|(thousand))th)")

In [ ]:
types_lieux = {"quartier","zone","lieu","espace","endroit","ville","localité","village","commune","agglomération","bourgade","cité","ghetto","arrondissement","terrain","surface"}
types_lieux = {"district","space","area","place","city","suburb","location","village","city","neighbourhood","turf","locality","sector"}
geos = {"france","europe","européen","hauts-de-seine","continental","continent","français","paris","parisien""européen","francilien","parisien","puteaux","nanterre","nanterrien","putéolien","courbevoie","courbevoisien","garenne-colombes","seine"}
geos= {"France","french","Europe","european","continent","continental","Paris","parisian","Nanterre","Puteaux","Courbevoie","Hauts-de-Seine","Île-de-France","Garenne-Colombes","Seine","west","western"}

In [ ]:
vecteur_lieu = moyenne_vecteurs(types_lieux)
vecteur_geo =  moyenne_vecteurs(geos)

12
16


In [ ]:
cos_sim(model_w2v["beacon"],vecteur_geo)

0.30446997

In [ ]:
cos_sim(model_w2v["valve"],model_w2v["Défense"])

0.12655103

In [ ]:
cos_sim(model_w2v["beacon"],vecteur_lieu)

0.428376

In [ ]:
def cree_features_mention(df):
  data = []
  for i  in df.index:
    mention = df.loc[i,"mention"]
    
    if i%10 == 0 :
      print(i)
    mention= normalise_mention(mention)
    is_norme = detecte_norme(mention)
    is_lieu,is_geo,dist_lieu,dist_geo,dist_norme = contient_lieu_geo(mention,vecteur_lieu,vecteur_geo)
    #dist_lieu,dist_geo,dist_norme = distances_procedes(mention,vecteur_lieu,vecteur_geo)
    data.append([is_norme,contient_ordinal(mention),contient_superlatif(mention),is_lieu,dist_lieu,is_geo,dist_geo,dist_norme,contient_guillemet(mention)])

  return pd.DataFrame(data,index=df.index,columns=["is_norme","contient_ordinal","contient_superlatif","contient_lieu","distance_lieu","contient_geo","distance_geo","distance_norme","contient_guillemet"])

1610
1620
1630
1640
La Défense in the west 1684
1690
La Défense in the west 1698
1700
1720
1760
LA DÉFENSE [DÉFENSE]
1770
1780
1810
1820
La Défense in the west central area of the city 1826
1830
1840
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
its new post-Brexit home 2005
2020
2030
2050
2060
2070
2080
2100
2120
2130
2140
2160
2170
Paris's safety valve for office space 2171
2180
2190
2200
2210


In [ ]:
df_features = cree_features_mention(df)

In [ ]:
df_features.to_csv(chemin+"/Corpus/features_EN_mentions_Defense.csv",sep=",")

In [ ]:
df_features

,is_norme,contient_ordinal,contient_superlatif,contient_lieu,distance_lieu,contient_geo,distance_geo,distance_norme,contient_guillemet
1610,0,0,0,0,0.230985,1,0.297212,0.469245,0
1611,0,0,0,0,0.230985,1,0.297212,0.469245,0
1616,0,0,0,0,0.230985,1,0.297212,0.469245,0
1618,0,0,0,1,0.730576,1,0.488771,0.239641,0
1620,0,0,0,1,0.730576,0,0.488771,0.239641,0
...,...,...,...,...,...,...,...,...,...
2206,0,0,0,1,0.730576,0,0.488771,0.239641,0
2208,1,0,0,0,0.301100,0,0.519467,1.000000,0
2209,0,0,0,1,0.730576,1,0.488771,0.239641,0
2210,1,0,0,0,0.301100,0,0.519467,1.000000,0
